In [3]:
from os import listdir, walk
from os.path import isfile, join
import numpy as np
from PIL import Image
import cv2
from IPython.display import display
import math, random
import time, datetime, sys, os, shutil
import operator

import tensorflow as tf
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Input
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.optimizers import *

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split
import pandas as pd

print(tf.config.experimental.list_physical_devices())

BATCH_SIZE = 32

%run ./variables.ipynb
%run ./utils.ipynb
%run ../utils/data_utils.ipynb

id_map = get_selected_taxons(SELECTED_TAXONS)
id_map_inv = get_selected_taxons(SELECTED_TAXONS, inv=True)
n_classes = len(list(id_map.keys()))
check_dirs(SAVED_MODELS_ROOT)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'), PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Handle data

## Preparing panda arrays

In [4]:
X_train, y_train, _ = get_dataset(TRAIN_DATASET_PATH, ids=False)
X_test, y_test, _ = get_dataset(TEST_DATASET_PATH, ids=False)

print("Test length:", len(X_train), "- n classes:", len(np.unique(y_train)))
print("Test length:", len(X_test), "- n classes:", len(np.unique(y_test)))

# Balance dataset
#X_train, y_train, max_samples = balance_dataset(X_train, y_train, max_samples=None)

'38/38'

'38/38'

Test length: 753 - n classes: 38
Test length: 84 - n classes: 38


In [5]:
data_train = {'png_path':  X_train, 'taxon_id': y_train}
data_test = {'png_path':  X_test, 'taxon_id': y_test}

df_train = pd.DataFrame(data_train, columns = ['png_path', 'taxon_id'])
df_test = pd.DataFrame(data_test, columns = ['png_path', 'taxon_id'])
df_train = df_train.sample(frac=1).reset_index(drop=True)
df_test = df_test.sample(frac=1).reset_index(drop=True)

# Prtining some infos
print(len(df_train), len(df_test))
df_train.head()
df_test.head()

753 84


,png_path,taxon_id
0,/mnt/nvme-storage/pfauregi/training/thumbails/...,Nitzschia_hantzschiana
1,/mnt/nvme-storage/pfauregi/training/thumbails/...,Encyonema_silesiacum
2,/mnt/nvme-storage/pfauregi/training/thumbails/...,Denticula_tenuis
3,/mnt/nvme-storage/pfauregi/training/thumbails/...,Diatoma_mesodon
4,/mnt/nvme-storage/pfauregi/training/thumbails/...,Nitzschia_dissipata


## Setting up the flows

In [6]:
train_datagen = ImageDataGenerator(rescale=1./255.,
                         rotation_range=20, 
                         brightness_range=[0.8,1.2], 
                         horizontal_flip=True, 
                         vertical_flip=True,
                         fill_mode='nearest',
                         width_shift_range=10,
                         height_shift_range=10,
                         zoom_range=0.2,
                         data_format="channels_last")

test_datagen = ImageDataGenerator(rescale=1./255.,
                         data_format="channels_last")

In [7]:
#classes_array = np.unique(y_train)
train_generator = train_datagen.flow_from_dataframe(
        dataframe=df_train,
        x_col='png_path',
        y_col='taxon_id',
        target_size=(256, 256),
        batch_size=32,
        #classes=classes_array,
        class_mode='categorical')
val_generator = test_datagen.flow_from_dataframe(
        dataframe=df_test,
        x_col='png_path',
        y_col='taxon_id',
        target_size=(256, 256),
        batch_size=32,
        classes=train_generator.class_indices,
        class_mode='categorical')

train_spe = train_generator.samples // BATCH_SIZE
val_spe = val_generator.samples // BATCH_SIZE

Found 753 validated image filenames belonging to 38 classes.
Found 84 validated image filenames belonging to 38 classes.


In [8]:
class_indices = train_generator.class_indices
f = open(os.path.join(SAVED_MODELS_ROOT, 'model_id_map.csv'), 'w')
with f:
    writer = csv.writer(f)
    writer.writerow(["taxon", "id"])
    for taxon in class_indices:
        writer.writerow([taxon, class_indices[taxon]])

## Testing

In [ ]:
i = 0
stop = False
for batch in train_generator:
    images = batch[0]
    labels = batch[1]
    for i in range(images.shape[0]):
        print(np.argmax(labels[i]))
        image = (images[i,:,:,:]*255).astype('uint8')
        #print(image)
        display(Image.fromarray(image))
        i+=1
        if i>=1:
            stop = True
            break
    if stop: pass;
    #display()

# Model desgin

In [9]:
# fetching base model
input_tensor = Input(shape=(256, 256, 3))
#base_model = InceptionV3(weights='imagenet', input_tensor=input_tensor, include_top=False)
base_model = Xception(include_top=False, weights='imagenet', input_tensor=input_tensor, pooling=None)

In [10]:
# setting model for specifiv case
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
out = Dense(len(train_generator.class_indices.keys()), activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=out)

# Training

## Setting callbacks

In [11]:
# Setting tensorboard
check_dirs(LOG_DIR)
delete_all_files_in_folder(LOG_DIR)
log_dir = LOG_DIR + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [12]:
log_file=os.path.join(SAVED_MODELS_ROOT, "model.log")
os.remove(log_file)
csv_logger = CSVLogger(log_file, append=True)

## Setting optimizer

In [17]:
sgd = SGD(lr=0.01, decay=0, momentum=0, nesterov=False)
optimizer = sgd

## New layers only

In [14]:
n_epochs = 5
last_epoch = get_last_epoch(log_file)

print(n_epochs, "epochs composed of", train_spe, "batches (steps) of", BATCH_SIZE, "images.")

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_generator, 
          epochs=last_epoch+n_epochs, 
          steps_per_epoch=train_spe,
          use_multiprocessing=False, 
          validation_data=val_generator,
          validation_steps=val_spe,
          callbacks=[tensorboard_callback, csv_logger],
          initial_epoch=last_epoch)

5 epochs composed of 23 batches (steps) of 32 images.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 23 steps, validate for 2 steps
Epoch 1/5
23/23 [==============================] - 13s 577ms/step - loss: 3.6202 - accuracy: 0.0499 - val_loss: 3.6204 - val_accuracy: 0.0469
Epoch 2/5
23/23 [==============================] - 10s 435ms/step - loss: 3.5669 - accuracy: 0.0874 - val_loss: 3.5913 - val_accuracy: 0.0469
Epoch 3/5
23/23 [==============================] - 10s 433ms/step - loss: 3.5118 - accuracy: 0.1650 - val_loss: 3.5661 - val_accuracy: 0.0781
Epoch 4/5
23/23 [==============================] - 10s 433ms/step - loss: 3.4632 - accuracy: 0.2136 - val_loss: 3.5410 - val_accuracy: 0.0938
Epoch 5/5
23/23 [==============================] - 10s 433ms/step - loss: 3.4159 - accuracy: 0.2732 - val_loss: 3.5171 - val_accuracy: 0.0938


In [25]:
np.max(history.history["val_accuracy"])

0.15625

## Training full model

In [18]:
n_epochs = 40
last_epoch = get_last_epoch(log_file)

print(n_epochs, "epochs composed of", train_spe, "batches (steps) of", BATCH_SIZE, "images.")

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers:
    layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
history = model.fit(train_generator, 
          epochs=last_epoch+n_epochs, 
          steps_per_epoch=train_spe,
          use_multiprocessing=False, 
          validation_data=val_generator,
          validation_steps=val_spe,
          callbacks=[tensorboard_callback, csv_logger],
          initial_epoch=last_epoch)
print(np.max(history.history["val_accuracy"]))

40 epochs composed of 23 batches (steps) of 32 images.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 23 steps, validate for 2 steps
Epoch 5/44
23/23 [==============================] - 13s 570ms/step - loss: 3.0754 - accuracy: 0.2580 - val_loss: 3.3005 - val_accuracy: 0.1719
Epoch 6/44
23/23 [==============================] - 10s 451ms/step - loss: 2.7112 - accuracy: 0.3689 - val_loss: 3.0045 - val_accuracy: 0.2812
Epoch 7/44
23/23 [==============================] - 10s 450ms/step - loss: 2.3380 - accuracy: 0.5257 - val_loss: 2.7297 - val_accuracy: 0.3750
Epoch 8/44
23/23 [==============================] - 10s 455ms/step - loss: 2.0184 - accuracy: 0.6644 - val_loss: 2.4394 - val_accuracy: 0.4219
Epoch 9/44
23/23 [==============================] - 10s 453ms/step - loss: 1.7401 - accuracy: 0.7531 - val_loss: 2.1527 - val_accuracy: 0.4844
Epoch 10/44
23/23 [==============================] - 10s 453ms/step - loss: 1.4643 - accuracy: 0.8155 - val_loss: 1.8553 - val_accuracy: 0.

## Saving model

In [13]:
save_model(model, SAVED_MODELS_ROOT)

Saved model to /mnt/nvme-storage/pfauregi/training/thumbails/ADIAC_D38/saved_models/model.json
Saved weights to /mnt/nvme-storage/pfauregi/training/thumbails/ADIAC_D38/saved_models/model.h5
